In [ ]:
#build a DinoSR model, save it, load the model to a new instance and check if the models are the same
from dinosr import DinoSR, model_creator
from dinosr_config import DinosrAudioConfig
from model_persistant_state import ModelPersistantState
import yaml
from dataset import get_dataloader
import torch

In [ ]:
model_path = "models/dummy"
!rm -rf {model_path}
mps = ModelPersistantState(model_path)

In [ ]:
# load from config.yaml

with open("config.yaml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)
cfg = DinosrAudioConfig(
    **cfg
)
model1 = DinoSR(cfg, model_creator)
model1.to('cuda')

dataloader = get_dataloader(10)

optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)

# train the model for 10 epochs
cnt = 0
for waveforms, lengths in dataloader:
    optimizer.zero_grad()
    result = model1(waveforms, lengths)
    loss = result["loss"]
    loss.backward()
    optimizer.step()
    cnt += 1
    if cnt > 10:
        break


# save the model
mps.save_model(
    0,
    model1,
    {
        "accuracy": 0.5,
        "loss": 0.5
    }
)


In [ ]:
model2 = mps.load_model().to('cuda')

In [ ]:
# compare the two model's state dicts
for (k1, v1), (k2, v2) in zip(model1.state_dict().items(), model2.state_dict().items()):
    print(k1)
    assert k1 == k2
    assert v1.equal(v2)

In [ ]:
# check if the codebooks are the same
for cb1, cb2 in zip(model1.codebook.codebooks, model2.codebook.codebooks):
    assert cb1.equal(cb2)

for cb1, cb2 in zip(model1.codebook.codebooks_sum, model2.codebook.codebooks_sum):
    assert cb1.equal(cb2)

for cb1, cb2 in zip(model1.codebook.codebooks_cnt, model2.codebook.codebooks_cnt):
    assert cb1.equal(cb2)

In [ ]:
model1.student

In [ ]:
cfg.encoder_layers

In [1]:
import torch
from codebook import Codebook

In [2]:
cb = Codebook(10, 100, 1)

In [5]:
cb.accumulate_codewords_for_update(0, torch.randn(50, 10).to('cuda'), torch.randint(0, 10, (50,)).to('cuda'))

In [6]:
print(cb.accumulated_x[0].shape)
print(cb.accumulated_closest_codewords[0].shape)

torch.Size([50, 10])
torch.Size([50])


In [7]:
cb.update_codewords()

torch.Size([50, 100])


In [2]:
import torch.nn.functional as F
import torch

In [5]:
# randomize a vector of size 10 and calculate its kl divergence from the uniform vector
x = torch.randn(1000, 10)
x = F.softmax(x, dim=-1)
x = x.mean(dim=0)
# x[0] = 0

y = F.softmax(torch.randn(10), dim=0)

kl = F.kl_div(x.log(), y, reduction='batchmean')
print(kl)

tensor(0.0685)


In [6]:
import torch

# randomize a vector of size 10 and a boolean vector of size 10
x = torch.randn(10)
mask = torch.randint(0, 2, (10,)).bool()
x[mask].shape[0]

5